In [35]:
import pandas as pd
import numpy as np
import joblib

In [36]:
var_map = {'Id':'Id',
 'MSSubClass':'Tipo de vivienda en venta',
 'MSZoning':'Zonificacion',
 'LotFrontage': 'Pies conectados a la propiedad',
 'LotArea': 'Tamaño del lote',
 'Street': 'Calle',
 'Alley': 'Pasillo',
 'LotShape': 'Forma',
 'LandContour': 'Planitud',
 'Utilities': 'Utilidades',
 'LotConfig': 'Configuracion del lote',
 'LandSlope':'Pendiente',
 'Neighborhood':'Colonia',
 'Condition1':'Proximidad a condiciones',
 'Condition2':'Proximidad a otras condiciones',
 'BldgType':'Tipo de vivienda',
 'HouseStyle':'Estilo',
 'OverallQual': 'Calidad',
 'OverallCond':'Condicion',
 'YearBuilt':'Año de construccion',
 'YearRemodAdd':'Año de remodelacion',
 'RoofStyle':'Estilo de techo',
 'RoofMatl':'Material del techo',
 'Exterior1st':'Revestimiento exterior',
 'Exterior2nd':'Otro revestimiento exterior',
 'MasVnrType':'Canteria',
 'MasVnrArea':'Area de canteria',
 'ExterQual':'Calidad de materiales exteriores',
 'ExterCond':'Condicion de materiales exteriores',
 'Foundation':'Cimentacion',
 'BsmtQual':'Altura de sotano',
 'BsmtCond':'Estado del sotano',
 'BsmtExposure':'Paredes de planta baja/jardin',
 'BsmtFinType1':'Califacion de superficie del sotano',
 'BsmtFinSF1':'Acabados tipo 1',
 'BsmtFinType2':'Otra califiacion de la superficie acabada del sotano',
 'BsmtFinSF2':'Acabados tipo 2',
 'BsmtUnfSF':'Superficie del sotano sin terminar',
 'TotalBsmtSF':'Superficie total del sotano',
 'Heating':'Tipo de calefaccion',
 'HeatingQC':'Calidad de calefaccion',
 'CentralAir':'Aire acondicionado central',
 'Electrical':'Sistema electrico',
 '1stFlrSF':'Primer piso',
 '2ndFlrSF':'Segundo piso',
 'LowQualFinSF':'Acabados de baja calidad',
 'GrLivArea':'Superficie habitable arriba del suelo',
 'BsmtFullBath':'Baños completos en el sotano',
 'BsmtHalfBath':'Medios baños en el sotano',
 'FullBath':'Baños completos arriba del suelo',
 'HalfBath':'Medios baños arriba del suelo',
 'BedroomAbvGr':'Recamaras arriba del suelo',
 'KitchenAbvGr':'Cocina arriba del suelo',
 'KitchenQual':'Calidad de cocina',
 'TotRmsAbvGrd':'Total de habitaciones arriba del suelo',
 'Functional':'Funcionalidad',
 'Fireplaces':'Chimeneas',
 'FireplaceQu': 'Calidad de chimenea',
 'GarageType':'Tipo de garage',
 'GarageYrBlt':'Año de construccion de garage',
 'GarageFinish':'Terminacion del garage',
 'GarageCars':'Capacidad de coches en el garage',
 'GarageArea':'Area del garage',
 'GarageQual':'Calidad de garage',
 'GarageCond':'Condicion del garage',
 'PavedDrive':'Calzada pavimentada',
 'WoodDeckSF':'Superficie de la cubierta',
 'OpenPorchSF':'Area de porche',
 'EnclosedPorch':'Area de porche cerrado',
 '3SsnPorch':'Area de porche de tres estaciones',
 'ScreenPorch':'Area del porche cubierto',
 'PoolArea':'Area de la piscina',
 'PoolQC':'Calidad de la piscina',
 'Fence':'Calidad de valla',
 'MiscFeature':'Elementos extra',
 'MiscVal':'Precio de elemento extra',
 'MoSold':'Mes de venta',
 'YrSold':'Año de venta',
 'SaleType':'Tipo de venta',
 'SaleCondition':'Condicion de venta',
 'SalePrice':'Precio de venta'}

In [37]:
df = pd.read_csv('train_clean.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
#df.rename(columns=var_map,inplace=True)
numeric_cols = df.select_dtypes(include=['number']).columns
categ_cols = df.select_dtypes(include=['object']).columns


In [ ]:
df.columns

In [39]:
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

In [40]:
#PREPROCESAMIENTO
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#Escalamiento para las variables numericas
#Dummies para las variables categoricas
preprocessing = ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_cols[:-1]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categ_cols)
    ])

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
modelo = GradientBoostingRegressor(criterion= 'squared_error', learning_rate= 0.2,
                                    loss= 'squared_error', n_estimators= 200) 
                                    #subsample= 0.1) #0.018
pipeline = Pipeline(steps=[('preprocessor', preprocessing),
                           ('model', modelo)])

In [42]:
'''
from sklearn.model_selection import GridSearchCV
# Definir los hiperparámetros que deseas ajustar y los valores posibles
param_grid = {
    'model__loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],  # Los tipos de perdida'squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’},  # Número de estimadores (árboles)
    'model__learning_rate': np.linspace(0.01, 0.1, 1),  # Tasa de aprendizaje
    'model__n_estimators': [50, 100, 200, 300, 500],  # NÃ®mero de estimadores,  # Profundidad máxima del árbol
    'model__subsample': np.linspace(0.1, 1, 1),  # Proporción de muestras en cada nodo,  # Mínimo de muestras para dividir un nodo
    'model__criterion': ['friedman_mse', 'squared_error']  # Mínimo de muestras en una hoja
}

# Realizar una búsqueda de hiperparámetros utilizando Grid Search Cross-Validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)  # X_train e y_train son tus datos de entrenamiento

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)
'''

'\nfrom sklearn.model_selection import GridSearchCV\n# Definir los hiperparámetros que deseas ajustar y los valores posibles\nparam_grid = {\n    \'model__loss\': [\'squared_error\', \'absolute_error\', \'huber\', \'quantile\'],  # Los tipos de perdida\'squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’},  # Número de estimadores (árboles)\n    \'model__learning_rate\': np.linspace(0.01, 0.1, 1),  # Tasa de aprendizaje\n    \'model__n_estimators\': [50, 100, 200, 300, 500],  # NÃ®mero de estimadores,  # Profundidad máxima del árbol\n    \'model__subsample\': np.linspace(0.1, 1, 1),  # Proporción de muestras en cada nodo,  # Mínimo de muestras para dividir un nodo\n    \'model__criterion\': [\'friedman_mse\', \'squared_error\']  # Mínimo de muestras en una hoja\n}\n\n# Realizar una búsqueda de hiperparámetros utilizando Grid Search Cross-Validation\ngrid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=\'neg_mean_squared_error\')\ngrid_search.fit(X, y)  # X_train e y_train s

In [43]:
pipeline.fit(X, y)
y_pred = pipeline.predict(X)
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

0.04632793120591925

In [44]:
#version = 1

Prueba: Leer test, llenar NA y hacer predicciones. Exportar a csv

In [45]:
test = pd.read_csv('test.csv')
X_ = test.drop(['Id'],axis=1)
y_ = test['Id']
media = X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].mean()
X_[['YearBuilt', 'YearRemodAdd','GarageYrBlt','YrSold']].fillna(media)
X_.fillna(0,inplace=True)
results = pipeline.predict(X_)

csv =pd.DataFrame({'Id':test['Id'], 'SalePrice': results})
csv.to_csv(f'predicciones/test_results_{version}.csv',index=False)
version += 1

KeyError: "['FirstFlrSF', 'SecondFlrSF', 'ThreeSsnPorch'] not in index"

In [ ]:
pipeline.feature_names_in_

array(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'PavedDriv

In [46]:
joblib.dump(pipeline,'pipeline_houses_final_v4.joblib')

['pipeline_houses_final_v4.joblib']

In [47]:
pipeline.feature_names_in_

array(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'FirstFlrSF', 'SecondFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
     

In [ ]:
import json
with open('casas.json', 'r') as archivo:
    casas_json = json.load(archivo)
# Convierte el JSON nuevamente en un DataFrame
df = pd.DataFrame(casas_json['casas'])
print(np.array(df.columns))

['MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'Utilities' 'LotConfig' 'LandSlope'
 'Neighborhood' 'Condition1' 'Condition2' 'BldgType' 'HouseStyle'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle'
 'RoofMatl' 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea'
 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType'
 'GarageYrBlt' 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual'
 'GarageCond' 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'PoolQC' 'Fence' 'MiscFeature'
 'MiscVal' 'MoSold' 'YrSold' 'SaleType' '

In [ ]:
df.iloc[-1]

MSSubClass           20
MSZoning             RL
LotFrontage        75.0
LotArea            9937
Street             Pave
                  ...  
MoSold                6
YrSold             2008
SaleType             WD
SaleCondition    Normal
SalePrice        147500
Name: 1378, Length: 80, dtype: object